In [1]:
import os
from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent
from os import getenv
from dotenv import load_dotenv
from prompts import RESEARCH_INSTRUCTION


load_dotenv(os.path.join("../../..", ".env"), override=True)

True

In [2]:
model = init_chat_model(
    model="nvidia/nemotron-3-nano-30b-a3b:free",
    model_provider="openai",
    base_url="https://openrouter.ai/api/v1",
    api_key=getenv("MODEL_API_KEY"),
)

agent = create_deep_agent(
    model=model,
    tools=[],
    system_prompt=RESEARCH_INSTRUCTION
)

In [57]:
result = agent.invoke({"messages": [{"role": "user", "content": "What is langgraph?"}]})

In [50]:
result

{'messages': [HumanMessage(content='What is langgraph?', additional_kwargs={}, response_metadata={}, id='ed858d21-a97c-4f8d-bb72-0623162a7a69'),
  AIMessage(content='**LangGraph** is an open‑source Python library designed to help developers build **stateful, multi‑agent, and workflow‑oriented applications** on top of large language models (LLMs). It was created by the team behind **LangChain**, and it extends the ideas of graph‑based execution to make it easier to coordinate complex sequences of LLM calls, tool usage, memory, and other side‑effects.\n\n### Key Concepts\n\n| Concept | What it Means |\n|---------|---------------|\n| **Graph‑based workflow** | You define a directed graph where nodes represent steps (e.g., LLM calls, Python code, external API calls) and edges represent the flow of data. This gives you a visual, programmable way to orchestrate complex pipelines. |\n| **Stateful agents** | Nodes can maintain and propagate state across multiple turns, enabling agents that rem

In [13]:
from sqlalchemy import create_engine, text
from langchain_community.utilities import SQLDatabase

conn_str = "mssql+pyodbc:///?odbc_connect=Driver={ODBC Driver 18 for SQL Server};Server=tcp:underwriting-accelerator-sqlserver.database.windows.net,1433;Database=free-sql-db-8814968;Uid=abhishek_admin;Pwd=ygN8dO15e9BBQ4a6;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=120"

engine = create_engine(conn_str, pool_pre_ping=True, pool_size=5)

db = SQLDatabase(engine)



In [4]:
from sqlalchemy import text

query = text("Select count(1) from Submissions")

v = db.run(query)

In [15]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [5]:
from prompts import SQL_ANALYST_AGENT_PROMPT

system_prompt = SQL_ANALYST_AGENT_PROMPT.format(
    dialect=db.dialect,
    top_k=5,
)


In [16]:
sqlagent = create_deep_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt
)

In [9]:
question = "Which is submission count?"

for step in sqlagent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which is submission count?
================================ Human Message =================================

Which is submission count?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_22d82ae47a7345a3831f4ff6)
 Call ID: call_22d82ae47a7345a3831f4ff6
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

SubmissionDocuments, Submissions, underwriting_rules
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_019a1bf50f71428992a84a6c)
 Call ID: call_019a1bf50f71428992a84a6c
  Args:
    table_names: Submissions
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE [Submissions] (
	[SubmissionID] UNIQUEIDENTIFIER NOT NULL DEFAULT (newid()), 
	[SubmissionNo] 

In [13]:
result=sqlagent.invoke( {"messages": [{"role": "user", "content": question}]})

In [21]:
result['messages'][-1].pretty_print()

================================== Ai Message ==================================

The **Submissions** table currently contains **3** records. Therefore, the submission count is **3**. If you were looking for a specific column named “submission count,” the table does not have one—so we infer the count by counting the rows, which yields 3.


In [17]:
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 

sqlagentwithinterupt = create_deep_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"sql_db_query": True}, 
            description_prefix="Tool execution pending approval", 
        ), 
    ], 
    checkpointer=InMemorySaver(), 
)

In [18]:
question = "Which is submission count?"
config = {"configurable": {"thread_id": "1"}} 

for step in sqlagentwithinterupt.stream(
    {"messages": [{"role": "user", "content": question}]},
    config, 
    stream_mode="values",
):
    if "__interrupt__" in step: 
        print("INTERRUPTED:") 
        interrupt = step["__interrupt__"][0] 
        for request in interrupt.value["action_requests"]: 
            print(request["description"]) 
    elif "messages" in step:
        step["messages"][-1].pretty_print()
    else:
        pass

================================ Human Message =================================

Which is submission count?
================================ Human Message =================================

Which is submission count?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_b7fde1fd2c1c400189a33ddd)
 Call ID: call_b7fde1fd2c1c400189a33ddd
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

SubmissionDocuments, Submissions, underwriting_rules
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_dc0e2106268b4cc0a2a82bec)
 Call ID: call_dc0e2106268b4cc0a2a82bec
  Args:
    table_names: Submissions
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE [Submissions] (
	[SubmissionID] UNIQUEIDENTIFIER NOT NULL DEFAULT (newid()), 
	[SubmissionNo] 

In [19]:
from langgraph.types import Command 

for step in sqlagentwithinterupt.stream(
    Command(resume={"decisions": [{"type": "approve"}]}), 
    config,
    stream_mode="values",
):
    if "messages" in step:
        step["messages"][-1].pretty_print()
    elif "__interrupt__" in step:
        print("INTERRUPTED:")
        interrupt = step["__interrupt__"][0]
        for request in interrupt.value["action_requests"]:
            print(request["description"])
    else:
        pass

================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_4c766a711ead4f48ace1eb7c)
 Call ID: call_4c766a711ead4f48ace1eb7c
  Args:
    query: SELECT COUNT(*) AS SubmissionCount FROM Submissions
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_4c766a711ead4f48ace1eb7c)
 Call ID: call_4c766a711ead4f48ace1eb7c
  Args:
    query: SELECT COUNT(*) AS SubmissionCount FROM Submissions
================================= Tool Message =================================
Name: sql_db_query

[(3,)]
================================== Ai Message ==================================

The table **Submissions** contains **3** records, so the submission count is **3**.
